In [1]:
import sys
sys.path.append('/Users/jodok/02 Code/EE-559-TEAM/Project1')
from src.dlc_practical_prologue import generate_pair_sets
from src.utils import load_class_data, load_target_data, load_all_data
from src.models import *
from src.trainer import Trainer
import matplotlib.pyplot as plt
from torch import nn

In [2]:
# Load data with class as target
dl_train_class, dl_val_class, dl_test_class = load_class_data()

# Load data with larger than as target
dl_train_target, dl_val_target, dl_test_target = load_target_data()

# Load data with class and larget than as targets
dl_train_all, dl_val_all, dl_test_all = load_all_data()

In [36]:
class TailNet(BaseModule):
    def __init__(self, lr=0.001):
        super().__init__(lr)
        self.conv1 = nn.Conv1d(1, 1, kernel_size=10, stride=10)
        self.fc1 = nn.Linear(2, 100)
        self.fc2 = nn.Linear(100, 10)
        self.fc3 = nn.Linear(10, 2)

    def forward(self, x):
        x = nn.functional.one_hot(x).float()
        x = x.view(-1, 1, 20)
        x = self.conv1(x)
        x = x.view(-1, 2)
        x = nn.functional.relu(x)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.fc2(x)
        x = nn.functional.relu(x)
        x = self.fc3(x)
        return x

    def training_step(self, batch, batch_idx):
        _, x, y = batch
        out = self(x)
        loss = self.loss(out, y)
        return loss
    
    def validation_step(self, batch, batch_idx):
        _, x, y = batch
        out = self(x)
        loss = self.loss(out, y)
        preds = torch.argmax(out, dim=1)
        acc = self.accuracy(preds, y)
        return loss, acc

class TestNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 1, kernel_size=10, stride=10)
    
    def forward(self, x):
        x = nn.functional.one_hot(x).float()
        x = x.view(-1, 1, 20)
        x = self.conv1(x)
        return x

In [37]:
tail = TailNet()
trainer = Trainer(nb_epochs=2)
trainer.fit(tail, dl_train_all, dl_val_all)

# Epoch 1/2:	 loss=0.69	 loss_val=0.69	 acc_val=56.7
# Epoch 2/2:	 loss=0.68	 loss_val=0.69	 acc_val=56.7


In [27]:
for batch in dl_train_all:
    _, x, y = batch
    break

In [28]:
test_net = TestNet()
out = test_net(x)

In [35]:
out.view(-1, 2).size()

torch.Size([32, 2])

In [20]:
torch.argmax(out, dim=2)
y

tensor([0, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0,
        0, 1, 0, 0, 1, 0, 1, 0])

In [12]:
x = nn.functional.one_hot(x, num_classes=10)
x.size()

torch.Size([32, 2, 10])

In [13]:
x.view(-1, 1, 20).float().size()

torch.Size([32, 1, 20])

In [23]:
nn.Flatten()(x).size()

torch.Size([32, 20])